In [1]:
import pandas as pd

In [23]:
pd.set_option('display.max_columns',None)

# 초중고등학교

In [21]:
school = pd.read_csv('../new_data/교육시설/청주대학교 지방교육재정연구원_초중등학교위치_20240322.csv', encoding='cp949')

In [94]:
tmp = school.loc[school['소재지지번주소'].str.contains('평택시')]
high = tmp.loc[tmp['학교급구분'] == '고등학교'][['학교명','소재지지번주소','위도','경도']]
high.rename(columns={'소재지지번주소':'주소'},inplace=True)

mid = tmp.loc[tmp['학교급구분'] == '중학교'][['학교명','소재지지번주소','위도','경도']]
mid.rename(columns={'소재지지번주소':'주소'},inplace=True)

ele = tmp.loc[tmp['학교급구분'] == '초등학교'][['학교명','소재지지번주소','위도','경도']]
ele.rename(columns={'소재지지번주소':'주소'},inplace=True)


# 학원

In [27]:
academy = pd.read_csv('../new_data/교육시설/학원교습소정보.csv', encoding='cp949', low_memory=False)

In [46]:
tmp2 = academy.loc[academy['시설도로명주소'].str.contains('평택시')]
aca = tmp2[['학원명','시설도로명주소']]

In [49]:
aca = aca.reset_index().drop(columns=['index'])

### 위도 경도 생성 (NAVER API)

In [51]:
import sys
sys.path.append("..")  # 상위 디렉토리를 sys.path에 추가

import requests
import pandas as pd
from mySetting import Client_ID, Client_Secret

# 네이버 API 키
client_id = Client_ID  # 클라이언트 ID
client_secret = Client_Secret  # 클라이언트 Secret

def get_coordinates(address):
    url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode'
    headers = {
        'X-NCP-APIGW-API-KEY-ID': client_id,
        'X-NCP-APIGW-API-KEY': client_secret,
    }
    params = {
        'query': address,
    }
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    
    if data['addresses']:
        x = data['addresses'][0]['x']
        y = data['addresses'][0]['y']
        return (x, y)
    else:
        return (None, None)

In [52]:
aca['위도'] = pd.NA
aca['경도'] = pd.NA

for i, address in enumerate(aca['시설도로명주소']):
    lng, lat = get_coordinates(address)
    aca.at[i, '위도'] = lat
    aca.at[i, '경도'] = lng

In [96]:
aca.rename(columns={'시설도로명주소':'주소'},inplace=True)
aca

,학원명,주소,위도,경도
0,1206보습학원,경기도 평택시 안중읍 안현로서8길 35,36.9793123,126.9218334
1,1618고등입시학원,경기도 평택시 중앙로 198,36.9909868,127.1048477
2,1719고등학원,경기도 평택시 경기대로 1523,37.0802809,127.0623642
3,531스페셜학원,경기도 평택시 만세로 1839,37.0044587,127.1038916
4,A1에듀(edu)학원,경기도 평택시 중앙로 207,36.9912544,127.1059930
...,...,...,...,...
1255,훈실용음악학원,경기도 평택시 용죽2로 32-82,36.9927389,127.1245561
1256,히어로애니미술학원,경기도 평택시 중앙로 183,36.9917687,127.1033724
1257,힐사이드컬리지잇주니어평택학원,경기도 평택시 죽백5로 41,37.0040274,127.1201783
1258,힘수학학원,경기도 평택시 용죽3로 19,36.9934509,127.1250668


# 어린이집

In [75]:
daycare = pd.read_csv('../new_data/교육시설/어린이집현황(제공표준).csv', encoding='cp949')

In [77]:
care = daycare.loc[daycare['소재지도로명주소'].str.contains('평택시')][['어린이집명','소재지도로명주소']].reset_index().drop(columns=['index'])

In [78]:
care['위도'] = pd.NA
care['경도'] = pd.NA

for i, address in enumerate(care['소재지도로명주소']):
    lng, lat = get_coordinates(address)
    care.at[i, '위도'] = lat
    care.at[i, '경도'] = lng

In [97]:
care.rename(columns={'소재지도로명주소':'주소'},inplace=True)
care

,어린이집명,주소,위도,경도
0,평택아너스어린이집,"경기도 평택시 평남로 671 104동 103호(비전동, 비전경남아너스빌아파트)",36.9938858,127.1099699
1,해링턴i어린이집,경기도 평택시 소사3로 22 효성해링턴플레이스 2단지 208동102호,36.9878033,127.1291729
2,효성다온어린이집,경기도 평택시 소사3로 22 효성해링턴플레이스 206동 104호,36.9878033,127.1291729
3,시립복창어린이집,경기도 평택시 신창로 12 (서정동),37.0747819,127.0425933
4,서정어린이집,경기도 평택시 서정로 205 (서정동),37.0665692,127.0561729
...,...,...,...,...
395,꼬마대통령어린이집,"경기도 평택시 평택5로 254 107동 106호(비전동, 한성아파트)",37.0003584,127.0982040
396,바다 어린이집,"경기도 평택시 죽백4로 16 102동 103호(죽백동, 더샵마스터뷰아파트)",37.0016641,127.1249822
397,소꿉놀이어린이집,"경기도 평택시 비전3로 59 810동 102호(비전동,엘에이치리더스하임아파트)",37.0056637,127.1064007
398,유보라키즈어린이집,경기도 평택시 비전2로 297 반도유보라 아이파크 104동 104호,37.0084575,127.1135559


# 유치원

In [81]:
kinder = pd.read_csv('../new_data/교육시설/유치원현황.csv', encoding='cp949')

In [90]:
kin = kinder.loc[kinder['정제지번주소'].str.contains('평택시')][['유치원명','정제지번주소','정제WGS84위도','정제WGS84경도']].dropna()

In [98]:
kin.rename(columns={'정제지번주소':'주소','정제WGS84위도':'위도','정제WGS84경도':'경도'},inplace=True)
kin

,유치원명,주소,위도,경도
796,새말유치원,경기도 평택시 칠원동 597번지,37.025258,127.108519
797,이화유치원,경기도 평택시 비전동 272-3번지,36.996373,127.104632
798,동화나라유치원,경기도 평택시 용이동 377번지,37.006730,127.138422
799,소사벌초등학교병설유치원,경기도 평택시 비전동 879-3번지,36.995313,127.116876
800,송신초등학교병설유치원,경기도 평택시 신장동 267번지,37.075197,127.052147
...,...,...,...,...
2169,아이들천국유치원,경기도 평택시 세교동 559번지 개나리아파트,37.001243,127.081848
2170,복창초등학교병설유치원,경기도 평택시 서정동 559-4번지,37.071232,127.050303
2171,안중초등학교병설유치원,경기도 평택시 안중읍 안중리 247-4번지,36.989649,126.926027
2172,지장초등학교병설유치원,경기도 평택시 서정동 824번지,37.068033,127.057600


In [100]:
high.to_csv('../preprocessed/교육시설/고등학교.csv', index=False)
mid.to_csv('../preprocessed/교육시설/중학교.csv', index=False)
ele.to_csv('../preprocessed/교육시설/초등학교.csv', index=False)
aca.to_csv('../preprocessed/교육시설/학원.csv', index=False)
kin.to_csv('../preprocessed/교육시설/유치원.csv', index=False)
care.to_csv('../preprocessed/교육시설/어린이집.csv', index=False)